In [1]:
import cv2
import numpy as np
img_path = 'box..jpg'
img = cv2.imread(img_path)
corX = []
corY = []
vanishing_points = []
real_distance = [0, 0, 0, 0]
colrgb = ['blue', 'green', 'red']

print ("Please Select the points of the Edges:")

Please Select the points of the Edges:


In [2]:
def mark_points(event, x, y, flags, param):
    global corX
    global corY
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(param[0], (int(x), int(y)), 4, param[1], -1)
        corX.append(x)
        corY.append(y)

In [3]:
#return vanishing points for x,y and z directions respectively.
def mark_edges(img):
    global corX
    global corY
    global vanishing_points
    for i in range(0, 3):
        print ("Mark 2 Points for ", colrgb[i],  " edge: ")
        color = [0, 0, 0]
        color[i] = 255
        color = tuple(color)
        cv2.namedWindow('select_edges')
        cv2.setMouseCallback('select_edges', mark_points, (img, color))
        while 1:
            cv2.imshow('select_edges', img)
            k = cv2.waitKey(20) & 0xFF
            if k == 27 or corX.__len__() == 6:
                cv2.imshow('select_edges', img)
                cv2.waitKey(20)
                break
            if k == ord("q"):
                exit(1)
                break

        cv2.line(img, (corX[0], corY[0]), (corX[1], corY[1]), color, 4)
        cv2.line(img, (corX[2], corY[2]), (corX[3], corY[3]), color, 4)
        cv2.line(img, (corX[4], corY[4]), (corX[5], corY[5]), color, 4)
        cv2.imshow('select_edges', img)
        cv2.waitKey(20)

        # Store the end-points
        e1 = [corX[0], corY[0], 1]
        e2 = [corX[1], corY[1], 1]
        e3 = [corX[2], corY[2], 1]
        e4 = [corX[3], corY[3], 1]

        # Parameters of lines
        L1 = np.cross(e1, e2)
        L2 = np.cross(e3, e4)

        # Intersection points = cross product of two lines
        V_homogenious = np.cross(L1, L2)

        # making the last coordinte 1 (homogenious co-ordinates)
        if V_homogenious[2] != 0:
            Vh = np.divide(V_homogenious, V_homogenious[2])
        vanishing_points.append(Vh)

        # for next side delete previous ones
        # print(corX)
        # print(corY)
        if i == 0:
            X_0 = corX
            Y_0 = corY
        elif i == 1:
            X_1 = corX
            Y_1 = corY
        else:
            X_2 = corX
            Y_2 = corY

        del corX[:]
        del corY[:]


    Vx = vanishing_points[0]
    Vy = vanishing_points[1]
    Vz = vanishing_points[2]
    print(Vx)
    print(Vy)
    print(Vz)
    return Vx,Vy,Vz


In [4]:
## Marking axes return worl coordinates and references points for axes
def mark_axes(img):
    global corX
    global corY
    print ("click on the origin and then the endpoints of the axes in the same order as you did previously")
    print ("also enter the distance from origin")
    for i in range(0, 4):
        color = [0, 0, 0]
        if i > 0:
            color[(i - 1) % 3] = 255
            color = tuple(color)
        cv2.namedWindow('select_axes')
        cv2.setMouseCallback('select_axes', mark_points, color)
        while 1:
            cv2.imshow('select_axes', img)
            k = cv2.waitKey(20) & 0xFF
            if k == 27 or corX.__len__() == (i + 1) :
                cv2.circle(img, (corX[i], corY[i]), 4, color, -1)
                if i != 0:
                    cv2.line(img, (corX[0], corY[0]), (corX[i], corY[i]), color, 4)
                cv2.imshow('select_axes', img)
                cv2.waitKey(20)
                break
            if k == ord("q"):
                exit(1)
                break

    return corX,corY  #corX[0] = wc, corX[1] = x direction ref point and similar for y and z


In [5]:
def projectionMatrix(vX, vY, vZ, xCor, yCor):
    w_origin = [xCor[0], yCor[0]]
    ref_1 = np.array([xCor[1], yCor[1]]) # x ref point
    ref_2 = np.array([xCor[2], yCor[2]]) # y ref point
    ref_3 = np.array([xCor[3], yCor[3]]) # z ref point

    x_length = np.sqrt(np.sum(np.square(ref_1 - w_origin)))   
    y_length = np.sqrt(np.sum(np.square(ref_2 - w_origin)))   
    z_length = np.sqrt(np.sum(np.square(ref_3 - w_origin)))   

    ref_1 = np.array(ref_1)
    ref_1 = np.expand_dims(np.append(ref_1, [1]), axis=0)
    ref_2 = np.array(ref_2)
    ref_2 = np.expand_dims(np.append(ref_2, [1]), axis=0)
    ref_3 = np.array(ref_3)
    ref_3 = np.expand_dims(np.append(ref_3, [1]), axis=0)
    w_origin = np.array(w_origin)
    w_origin = np.append(w_origin, [1])
    Vx = np.array(vX)
    Vy = np.array(vY)
    Vz = np.array(vZ)
    # print(Vx.shape, ref_1.shape, w_origin.shape) #(3,) (2,) (2,)
    # print("Vx, ref_1, w_origin")
    print(Vx, ref_1, w_origin)
    ax,resid,rank,s = np.linalg.lstsq( (Vx-ref_1).T , (ref_1 - w_origin).T, rcond=None )
    ax = ax[0][0]/x_length

    ay,resid,rank,s = np.linalg.lstsq( (Vy-ref_2).T , (ref_2 - w_origin).T, rcond=None )
    ay = ay[0][0]/y_length

    az,resid,rank,s = np.linalg.lstsq( (Vz-ref_3).T , (ref_3 - w_origin).T, rcond=None )
    az = az[0][0]/z_length

    px = ax*Vx
    py = ay*Vy
    pz = az*Vz

    P = np.empty([3,4]) #Projection Matrix
    P[:,0] = px
    P[:,1] = py
    P[:,2] = pz
    P[:,3] = w_origin


    Hxy = np.zeros((3,3))
    Hyz = np.zeros((3,3))
    Hzx = np.zeros((3,3))

    Hxy[:,0] = px
    Hxy[:,1] = py
    Hxy[:,2] = w_origin

    Hyz[:,0] = py
    Hyz[:,1] = pz
    Hyz[:,2] = w_origin

    Hzx[:,0] = px
    Hzx[:,1] = pz
    Hzx[:,2] = w_origin


    Hxy[0,2] = Hxy[0,2] 
    Hxy[1,2] = Hxy[1,2]

    Hyz[0,2] = Hyz[0,2] + 100
    Hyz[1,2] = Hyz[1,2] + 100 

    Hzx[0,2] = Hzx[0,2] - 50
    Hzx[1,2] = Hzx[1,2] + 50

    return Hxy, Hyz, Hzx

In [6]:
def getTextures(Txy, Tyz, Tzx):
  r,c,temp = img.shape
  Txy = cv2.warpPerspective(img,Hxy,(r,c),flags=cv2.WARP_INVERSE_MAP)
  Tyz = cv2.warpPerspective(img,Hyz,(r,c),flags=cv2.WARP_INVERSE_MAP)
  Tzx = cv2.warpPerspective(img,Hzx,(r,c),flags=cv2.WARP_INVERSE_MAP)

  cv2.imshow("Txy",Txy)
  cv2.imshow("Tyz",Tyz)
  cv2.imshow("Tzx",Tzx)

  cv2.waitKey(0)
  cv2.imwrite("XY.jpg",Txy)
  cv2.imwrite("YZ.jpg",Tyz)
  cv2.imwrite("ZX.jpg",Tzx)
  cv2.destroyAllWindows();
  print("textures saved")

In [7]:
Vx, Vy, Vz = mark_edges(cv2.imread('box..jpg'))
xCor, yCor = mark_axes(cv2.imread('box..jpg'))
print("Vx, Vy, Vz =", Vx,Vy, Vz)
print("xCor =", xCor)
print("yCor =", yCor)

Mark 2 Points for  blue  edge: 
Mark 2 Points for  green  edge: 
Mark 2 Points for  red  edge: 
[ 2.48533224e+03 -1.40656233e+02  1.00000000e+00]
[-472.65616438 -137.65890411    1.        ]
[ 6.98294118e+02 -5.71846471e+04  1.00000000e+00]
click on the origin and then the endpoints of the axes in the same order as you did previously
also enter the distance from origin
Vx, Vy, Vz = [ 2.48533224e+03 -1.40656233e+02  1.00000000e+00] [-472.65616438 -137.65890411    1.        ] [ 6.98294118e+02 -5.71846471e+04  1.00000000e+00]
xCor = [180, 373, 89, 177]
yCor = [346, 303, 275, 119]


In [8]:
Hxy, Hyz, Hzx = projectionMatrix(Vx, Vy, Vz, xCor, yCor)
# print(Hxy, Hyz, Hzx)
getTextures(Hxy, Hyz, Hzx)

[ 2.48533224e+03 -1.40656233e+02  1.00000000e+00] [[373 303   1]] [180 346   1]
textures saved
